# Layers

gdsfactory includes a generic example Technology in `gdsfactory.tech` that you can use as an inspiration to create your own.


Each foundry uses different GDS numbers for each process step.

We follow the generic layer numbers from the book "Silicon Photonics Design: From Devices to Systems Lukas Chrostowski, Michael Hochberg".

| GDS (layer, purpose) | layer_name | Description                                                 |
| -------------------- | ---------- | ----------------------------------------------------------- |
| 1 , 0                | WG         | 220 nm Silicon core                                         |
| 2 , 0                | SLAB150    | 150nm Silicon slab (70nm shallow Etch for grating couplers) |
| 3 , 0                | SLAB90     | 90nm Silicon slab (for modulators)                          |
| 4, 0                 | DEEPTRENCH | Deep trench                                                 |
| 47, 0                | MH         | heater                                                      |
| 41, 0                | M1         | metal 1                                                     |
| 45, 0                | M2         | metal 2                                                     |
| 40, 0                | VIAC       | VIAC to contact Ge, NPP or PPP                              |
| 44, 0                | VIA1       | VIA1                                                        |
| 46, 0                | PADOPEN    | Bond pad opening                                            |
| 51, 0                | UNDERCUT   | Undercut                                                    |
| 66, 0                | TEXT       | Text markup                                                 |
| 64, 0                | FLOORPLAN  | Mask floorplan                                              |

Layers are available in `gf.LAYER` as `gf.LAYER.WG`, `gf.LAYER.WGCLAD`


![layer stack](https://i.imgur.com/GUb1Kav.png)


A GDS has different layers to describe the different fabrication process steps.

GDS layers have 2 integer numbers: GDSlayer, GDSpurpose

Klayout shows Layers with a color, style and transparency when showing GDS layouts.

In [ ]:
import gdsfactory as gf

In [ ]:
c = gf.layers.LAYER_COLORS.preview()
c

In [ ]:
layer_wg = gf.LAYER.WG
print(layer_wg)

## Extract layers

You can also extract layers using the `extract` function. This function returns a new flattened Component that contains the extracted layers.
A flat Component does not have references, and all the polygons are absorbed by the top cell.

In [ ]:
extract = c.extract(layers=(gf.LAYER.M1, gf.LAYER.VIAC))
extract

## Remove layers

You can remove layers using the `remove_layers()` function.

In [ ]:
removed = extract.remove_layers(layers=(gf.LAYER.VIAC,))
removed

## Remap layers

You can remap (change the polygons from one layer to another layer) using the `remap_layer`, which will return a new `Component`

In [ ]:
c = gf.components.straight(layer=(2, 0))
c

In [ ]:
remap = c.remap_layers(layermap={(2, 0): gf.LAYER.WGN})
remap

## LayerStack

Each layer also includes the information of thickness and position of each layer.

This LayerStack can be used for:

- render a 3D image
- FDTD simulations

In [ ]:
c = gf.layers.LAYER_COLORS.preview()
c

In [ ]:
from gdsfactory.tech import LayerStack, LayerLevel, LAYER


def get_layer_stack_generic(
    thickness_silicon_core: float = 220e-3, thickness_cladding: float = 3.0
) -> LayerStack:
    """Returns generic LayerStack.
    based on paper https://www.degruyter.com/document/doi/10.1515/nanoph-2013-0034/html
    """
    return LayerStack(
        layers=dict(
            core=LayerLevel(
                layer=LAYER.WG,
                thickness=thickness_silicon_core,
                zmin=0.0,
                material="si",
            ),
            clad=LayerLevel(
                layer=LAYER.WGCLAD,
                thickness=thickness_cladding,
                zmin=0.0,
                material="sio2",
            ),
            slab150=LayerLevel(
                layer=LAYER.SLAB150,
                thickness=150e-3,
                zmin=0,
                material="si",
            ),
            slab90=LayerLevel(
                layer=LAYER.SLAB90,
                thickness=90e-3,
                zmin=0.0,
                material="si",
            ),
            nitride=LayerLevel(
                layer=LAYER.WGN,
                thickness=350e-3,
                zmin=220e-3 + 100e-3,
                material="sin",
            ),
            ge=LayerLevel(
                layer=LAYER.GE,
                thickness=500e-3,
                zmin=thickness_silicon_core,
                material="ge",
            ),
            via_contact=LayerLevel(
                layer=LAYER.VIAC,
                thickness=1100e-3,
                zmin=90e-3,
                material="Aluminum",
            ),
            metal1=LayerLevel(
                layer=LAYER.M1,
                thickness=750e-3,
                zmin=thickness_silicon_core + 1100e-3,
                material="Aluminum",
            ),
            heater=LayerLevel(
                layer=LAYER.HEATER,
                thickness=750e-3,
                zmin=thickness_silicon_core + 1100e-3,
                material="TiN",
            ),
            viac=LayerLevel(
                layer=LAYER.VIA1,
                thickness=1500e-3,
                zmin=thickness_silicon_core + 1100e-3 + 750e-3,
                material="Aluminum",
            ),
            metal2=LayerLevel(
                layer=LAYER.M2,
                thickness=2000e-3,
                zmin=thickness_silicon_core + 1100e-3 + 750e-3 + 1.5,
                material="Aluminum",
            ),
        )
    )


layer_stack220 = get_layer_stack_generic(thickness_silicon_core=0.220)

c = gf.components.straight_heater_metal(length=40)
c

In [ ]:
scene = c.to_3d(layer_stack=layer_stack220)
scene.show()

In [ ]:
# lets assume we have 9um silicon instead of 0.220um, You will see a much thicker waveguide under the metal heater.
layer_stack9000 = get_layer_stack_generic(thickness_silicon_core=9.0)
scene = c.to_3d(layer_stack=layer_stack9000)
scene.show()

## 3D rendering

To render components in 3D you will need to define two things:

1. LayerStack: for each layer contains thickness of each material and z position
2. LayerColors: for each layer contains colors (this file is the same that Klayout uses). You can load it with `gf.layers.load_lyp()`

In [ ]:
heater = gf.components.straight_heater_metal(length=50)
heater

In [ ]:
scene = heater.to_3d()
scene.show()

## 2.5D Klayout view

You can also get the script in KLayout 2.5D view.

In [ ]:
from gdsfactory.tech import LAYER_STACK

LAYER_STACK.get_klayout_3d_script()

Then you go go Tools → Manage Technologies


![klayout](https://i.imgur.com/KCcMRBO.png)

and Paste the script

![paste](https://i.imgur.com/CoTythB.png)


## Klayout cross-section

You can also install the [KLayout cross-section plugin](https://gdsfactory.github.io/klayout_pyxs/README.html)

![xsection](https://i.imgur.com/xpPS8fM.png)

and customize the cross_section script that you need to use for your technology.

In [ ]:
import pathlib
from gdsfactory.geometry.get_xsection_script import get_xsection_script
from gdsfactory.tech import LAYER


nm = 1e-3
if __name__ == "__main__":
    script = get_xsection_script(
        t_box=2.0,
        t_slab=110 * nm,
        t_si=220 * nm,
        t_ge=400 * nm,
        t_nitride=400 * nm,
        h_etch1=0.07,
        h_etch2=0.06,
        h_etch3=0.09,
        t_clad=0.6,
        t_m1=0.5,
        t_m2=0.5,
        t_m3=2.0,
        gap_m1_m2=0.6,
        gap_m2_m3=0.3,
        t_heater=0.1,
        gap_oxide_nitride=0.82,
        t_m1_oxide=0.6,
        t_m2_oxide=2.0,
        t_m3_oxide=0.5,
        layer_wg=LAYER.WG,
        layer_fc=LAYER.SLAB150,
        layer_rib=LAYER.SLAB90,
        layer_n=LAYER.N,
        layer_np=LAYER.NP,
        layer_npp=LAYER.NPP,
        layer_p=LAYER.P,
        layer_pp=LAYER.PP,
        layer_ppp=LAYER.PPP,
        layer_PDPP=LAYER.GEP,
        layer_nitride=LAYER.WGN,
        layer_Ge=LAYER.GE,
        layer_GePPp=LAYER.GEP,
        layer_GeNPP=LAYER.GEN,
        layer_viac=LAYER.VIAC,
        layer_viac_slot=LAYER.VIAC,
        layer_m1=LAYER.M1,
        layer_mh=LAYER.HEATER,
        layer_via1=LAYER.VIA1,
        layer_m2=LAYER.M2,
        layer_via2=LAYER.VIA2,
        layer_m3=LAYER.M3,
        layer_open=LAYER.PADOPEN,
    )

    # script_path = pathlib.Path(__file__).parent.absolute() / "xsection_planarized.pyxs"
    # script_path.write_text(script)
    print(script)

![xsection generic](https://i.imgur.com/H5Qiygc.png)